In [ ]:
import pandas as pd
import urllib.request
import json

# Evaluate Data Portals

In [ ]:
NUM_OF_MANUALLY_ADDED = 3
FILTER_TOP = 503 - NUM_OF_MANUALLY_ADDED # select the top N portals

In [ ]:
with open('../input/api.lab.key', 'r') as f:
    apiKey = f.read() # WAVE API key

In [ ]:
dbs = pd.read_json('../input/database-commons.json') # already sorted by impact scores
dbs = pd.DataFrame.from_dict(dbs.data.to_dict(), orient='index')
dbs = dbs[0:FILTER_TOP]
dbs

In [ ]:
manually_picked = pd.read_json('../input/manually-added-data-portals.json')
manually_picked
dbs = dbs.append(manually_picked)
dbs

In [ ]:
"""
Load existing reports
"""
f = open('../output/a11y-reports.json', 'r')
reports = json.load(f)
f.close()

"""
Iterate to add missing reports
"""
for index, row in dbs.iterrows():
    dbId = row.dbId
    shortName = row.shortName
    url = row.url

    reportExist = any(report['dbId'] == dbId for report in reports)
    if not reportExist:
        print('Loading... ', dbId, shortName, url)

        apiUrl = f'https://wave.webaim.org/api/request?key={apiKey}&reporttype=2&url={url}'
        # apiUrl = 'https://raw.githubusercontent.com/gosling-lang/gosling.js/master/tsconfig.json'
        with urllib.request.urlopen(apiUrl) as f:
            newReport = json.load(f)
            newData = {}
            newData['dbId'] = dbId
            newData['shortName'] = shortName
            newData['url'] = url
            newData['report'] = newReport
            reports.append(newData)
            # print(reports)

        with open('../output/a11y-reports.json', 'w') as f:
            json.dump(reports, f)